In [7]:
# import packages and libraries
import pandas as pd
import lightgbm
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import uuid
import os
from sklearn.metrics import r2_score
from datetime import datetime
init_notebook_mode(connected=True)

In [8]:
import sklearn
import modAL
sklearn
lightgbm.__version__
#modAL

'2.2.3'

In [13]:
###help functions###
def PlotCVRes_plotly(X, y, model, cv, scoring, titel):
    dir = os.path.dirname(os.path.abspath("__file__"))
    id = uuid.uuid4().hex
    if titel == '': 
        titel = str(model).replace('\n', '').replace('  ', '')
        titel_new = ''
        pos=0
        com_positions = pd.DataFrame([i for i, ltr in enumerate(titel) if (ltr == ',')])
        split_positions = [com_pos[0] for ind, com_pos in com_positions.iterrows() if ((ind+1)%5==0)] + [len(titel)]
        for ind in split_positions:
            titel_new = titel_new + ' \n ' + titel[pos:ind]
            pos = ind
        titel = titel_new[3:]

    cv_scores = cross_validate(model, np.array(X), np.array(y), cv=cv, scoring=scoring)
    print('MAE_test_mean: ' + str(round(abs(cv_scores['test_MAE']).mean(), 2)) + ', MAE_test_std: ' + str(
        round(abs(cv_scores['test_MAE']).std(), 2)))
    print('RMSE_test_mean: ' + str(round(np.sqrt(abs(cv_scores['test_RMSE'])).mean(), 2)) + ', RMSE_test_std: ' + str(
        round(np.sqrt(abs(cv_scores['test_RMSE'])).std(), 2)))
    print('R2_test_mean: ' + str(round(cv_scores['test_R2'].mean(), 3)) + ', R2_test_std: ' + str(
        round(cv_scores['test_R2'].std(), 3)))
    
    predicted = cross_val_predict(model, X, y, cv=cv)
    #predicted/real plot
    trace1 = go.Scatter(
        x = [y.min(), y.max()],
        y = [y.min(), y.max()],
        name = 'Ideal prediction line',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 3,
            dash = 'dash')
    )
    trace2 = go.Scattergl(
            x = y,
            y = predicted,
            name = 'Model Predictions',
            mode = 'markers',
            marker= dict(size= 3,
                opacity= 0.7,
                color = ('rgb(0, 102, 204)')
               ),
            #text=item_pairs,
            #name=orders,
            #hoverinfo = 'all', #"text + name",
            )

    data = [trace2, trace1]
    
    
    layout = go.Layout(
        xaxis=dict(
            title= 'Real Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        yaxis=dict(
            title= 'Predicted Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        # title="ML Model Cross-validation Results",
        font=dict(family='arial', size=20),
        autosize=False,
        width=600,
        height=600,
        showlegend=False,
        hovermode = "closest",
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, 'pred_real_scatter')

def Model_validation(X, y, model, scoring, data_stand=False, cv=5, plotCV=True, titel=''):
    if data_stand:
        Xs = StandardScaler().fit_transform(X)
    else:
        Xs = X
    # Plotting Cross-Validated Predictions
    if plotCV:
        PlotCVRes_plotly(Xs, y, model, cv, scoring, titel)
    else:
        cv_scores = cross_validate(model, Xs, y, cv=cv, scoring=scoring)
        print('MAE_test_mean: ' + str(abs(cv_scores['test_MAE']).mean()) + ', MAE_test_std: ' + str(
            abs(cv_scores['test_MAE']).std()))
        print('RMSE_test_mean: ' + str(np.sqrt(abs(cv_scores['test_RMSE'])).mean()) + ', RMSE_test_std: ' + str(
            np.sqrt(abs(cv_scores['test_RMSE'])).std()))
        print('R2_test_mean: ' + str(cv_scores['test_R2'].mean()) + ', R2_test_std: ' + str(
            cv_scores['test_R2'].std()))

In [10]:
data_path_ = r"..\..\01_Data\El_motor_Temp\pmsm_temperature_data.csv"
data = pd.read_csv(data_path_)
data.stator_yoke.min()
data.stator_yoke.max()
data.profile_id.unique()

array([ 4,  6, 10, 11, 20, 27, 29, 30, 31, 32, 36, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
       64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81,
       72], dtype=int64)

In [11]:
data[data.profile_id==66].shape

(36475, 13)

In [14]:
# set specific paths
scoring = {'R2': 'r2', 'RMSE': 'neg_mean_squared_error', 'MAE': 'neg_mean_absolute_error'}

data_path_ = r"..\..\01_Data\El_motor_Temp\pmsm_temperature_data.csv"
data = pd.read_csv(data_path_)
data_filt = data[data.profile_id==66]

print(data.head())
X, y = data_filt.iloc[::5 ,:8], data_filt.iloc[::5 , 8]
print(X.shape)
# Shuffel data
np.random.seed(1337)
rng_state = np.random.get_state()
X = np.random.permutation(X)
np.random.set_state(rng_state)
y = np.random.permutation(y)
y=y.ravel()

lgbmParams = {'boosting_type': 'gbdt',
                'n_estimators': 2500,
                'objective': 'regression',
                'learning_rate': 0.1,
                'max_depth': 8,
                'num_leaves': 30,
                'n_jobs': -1
              }

lgbmParams = {'boosting_type': 'gbdt',
              'random_state': 736283,
              'num_leaves': 400,
              'min_data_in_leaf': 5,
              'max_bin': 500,
              'learning_rate': 0.08,
              'n_estimators': 771,
              'n_jobs': -1,
              'subsample': 0.8
              }

lgbmParams = {'boosting_type': 'gbdt',
              'random_state': 736283,
              'num_leaves': 40,
              'min_data_in_leaf': 5,
              'max_bin': 500,
              'learning_rate': 0.08,
              'n_estimators': 77,
              'n_jobs': -1,
              'subsample': 0.8
              }

lgbmParams = {'learning_rate': 0.04477772796588966, 
              'n_estimators': 581, 
              'num_leaves': 84, 
              'n_jobs': -1,
              'objective': 'regression'}

model = lightgbm.LGBMRegressor(**lgbmParams)

# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(bootstrap=True, criterion='mse',
#                                        max_depth=None,
#                                        max_features=0.9000000000000001,
#                                        max_leaf_nodes=None,
#                                        min_impurity_decrease=0.0,
#                                        min_impurity_split=None,
#                                        min_samples_leaf=11, min_samples_split=9,
#                                        min_weight_fraction_leaf=0.0,
#                                        n_estimators=100, n_jobs=None,
#                                        oob_score=False, random_state=None,
#                                        verbose=0, warm_start=False)

# xgbParams = {'learning_rate': 0.024,
#              'n_estimators': 4900,
#              'max_depth': 6,
#              'missing': -1,
#              'tree_method': 'hist',
#              'min_child_weight': 1,
#              'gamma': 0,
#              'subsample': 0.7,
#              'colsample_bytree': 0.6,
#              'reg_alpha': 1,
#              'scale_pos_weight': 1,
#              'random_state': 736283,
#              'n_jobs': -1}
# model = XGBRegressor(**xgbParams)

xgbParams = {'learning_rate': 0.045, 
              'n_estimators': 581, 
              'n_jobs': -1}
model = XGBRegressor(**xgbParams)

# xgbParams = {'learning_rate': 0.045, 
#               'n_estimators': 581, 
#               'n_jobs': -1}
# model = XGBRegressor(**xgbParams)



Model_validation(X, y, model, scoring=scoring,
                        cv=3, data_stand=False, 
                        plotCV=True, titel='')

    ambient   coolant       u_d       u_q  motor_speed    torque       i_d  \
0 -0.752143 -1.118446  0.327935 -1.297858    -1.222428 -0.250182  1.029572   
1 -0.771263 -1.117021  0.329665 -1.297686    -1.222429 -0.249133  1.029509   
2 -0.782892 -1.116681  0.332771 -1.301822    -1.222428 -0.249431  1.029448   
3 -0.780935 -1.116764  0.333700 -1.301852    -1.222430 -0.248636  1.032845   
4 -0.774043 -1.116775  0.335206 -1.303118    -1.222429 -0.248701  1.031807   

        i_q        pm  stator_yoke  stator_tooth  stator_winding  profile_id  
0 -0.245860 -2.522071    -1.831422     -2.066143       -2.018033           4  
1 -0.245832 -2.522418    -1.830969     -2.064859       -2.017631           4  
2 -0.245818 -2.522673    -1.830400     -2.064073       -2.017343           4  
3 -0.246955 -2.521639    -1.830333     -2.063137       -2.017632           4  
4 -0.246610 -2.521900    -1.830498     -2.062795       -2.018145           4  
(7295, 8)
[20:23:40] WARNING: src/objective/regression_ob

In [142]:
model.fit(X, y)

data_filt_test = data[data.profile_id==66]
X_test, y_test = np.array(data_filt_test.iloc[:, :8]), np.array(data_filt_test.iloc[:, 8]).ravel()
y_pred_test = model.predict(X_test)
r2_score(y_test, y_pred_test)

[11:22:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


-1.4394620200396968

# Save model

In [146]:
import pickle
model.fit(X, y)
str_mdl = pickle.dumps(model, 0) # dump to model in a string
file_object = open('ElctrMotorTemp_ML.txt', 'wb')
file_object.write(str_mdl)

#Read models from string
file_object = open('ElctrMotorTemp_ML.txt', 'rb')
str_mdl = file_object.read()
model = pickle.loads(str_mdl)

[11:23:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [147]:
#Pickle the model
model.fit(X, y)
# from joblib import dump, load
# dump(model, 'PowerPlant_ML.joblib')
# import pickle
# pickled_mdl = pickle.dumps(model, 0).decode()  # dump to model in a string
# restored_mdl = pickle.loads(pickled_mdl.encode())  #read str object to ml model

[11:23:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.045, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=581,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [152]:
data_filt_test.describe().loc[['min', 'max'], :]

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
min,-4.626428,-1.169556,-1.649274,-1.527701,-1.222442,-3.301562,-2.689756,-3.295509,-0.843100,-1.457505,-1.655359,-1.660462,66.0
max,0.723836,2.287928,2.269332,1.793498,2.011761,2.662138,1.029197,2.664809,1.964513,2.416686,2.302169,2.158104,66.0


In [154]:
model.fit(X, y)
#validate on the random predictions from this range
# actual_lows = {'ambient': -8.5, 'coolant': -1.4, 'u_d': -1.65, 'u_q': -1.8, 'motor_speed': -1.37,
#               'torque': -3.3, 'i_d': -3.2, 'i_q': -3.3}  #, 'pm':[-2.6]}
# actual_highs = {'ambient': 2.9, 'coolant': 2.6, 'u_d': 2.27, 'u_q': 1.79, 'motor_speed': 2.02,
#               'torque': 3, 'i_d': 1.06, 'i_q': 2.9 }  #, 'pm':[2.9]}
# variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']

actual_lows = {'ambient': -4.62, 'coolant': -1.16, 'u_d': -1.64, 'u_q': -1.52, 'motor_speed': -1.22,
              'torque': -3.30, 'i_d': -2.68, 'i_q': -3.29}  #, 'pm':[-2.6]}
actual_highs = {'ambient': 0.72, 'coolant': 2.28, 'u_d': 2.26, 'u_q': 1.79, 'motor_speed': 2.02,
              'torque': 2.66, 'i_d': 1.03, 'i_q': 2.66 }  #, 'pm':[2.9]}
variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']


samples_ = 1000


def random_generator(samples_, actual_lows, actual_highs, variables_, model):
    df_doe = pd.DataFrame(columns=variables_[:-1])
    for var in variables_[:-1]:
        df_doe[var] = np.random.uniform(actual_lows[var], actual_highs[var],
                                        samples_).round(3)
    try:
        df_doe[variables_[-1]] = model.predict(np.array(df_doe)).round(3)
    except:
        df_doe[variables_[-1]] = model.predict(df_doe).round(3)
    return df_doe


df = random_generator(samples_, actual_lows, actual_highs, variables_, model)
print(df['pm'].min(), df['pm'].max())

[11:28:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-0.673 1.47


In [161]:
from pyDOE import lhs
actual_lows = {'ambient': -8.5, 'coolant': -1.4, 'u_d': -1.65, 'u_q': -1.8, 'motor_speed': -1.37,
              'torque': -3.3, 'i_d': -3.2, 'i_q': -3.3}  #, 'pm':[-2.6]}
actual_highs = {'ambient': 2.9, 'coolant': 2.6, 'u_d': 2.27, 'u_q': 1.79, 'motor_speed': 2.02,
              'torque': 3, 'i_d': 1.06, 'i_q': 2.9 }  #, 'pm':[2.9]}
variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']

# actual_lows = {'ambient': -4.62, 'coolant': -1.16, 'u_d': -1.64, 'u_q': -1.52, 'motor_speed': -1.22,
#               'torque': -3.30, 'i_d': -2.68, 'i_q': -3.29}  #, 'pm':[-2.6]}
# actual_highs = {'ambient': 0.72, 'coolant': 2.28, 'u_d': 2.26, 'u_q': 1.79, 'motor_speed': 2.02,
#               'torque': 2.66, 'i_d': 1.03, 'i_q': 2.66 }  #, 'pm':[2.9]}
# variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']
samples_ = 100

# def random_generator(samples_, actual_lows, actual_highs, variables_, model):
#     df_doe = pd.DataFrame(columns=variables_[:-1])
#     for var in variables_[:-1]:
#         df_doe[var] = np.random.uniform(actual_lows[var], actual_highs[var],
#                                         samples_).round(3)
#     df_doe[variables_[-1]] = model.predict(df_doe).round(2)
#     return df_doe

# def latin_hypercube_generator(samples_, actual_lows, actual_highs, variables, model):
#     df_doe = pd.DataFrame(lhs(len(variables) - 1, samples=samples_, criterion='maximin'))
#     df_doe.columns = variables[:-1]
#     for col in df_doe.columns:
#         df_doe[col] = [actual_lows[col]] * df_doe.shape[0] + df_doe[col] * (actual_highs[col] - actual_lows[col])
#         df_doe[col] = df_doe[col].apply(lambda x: round(x, 3))
#     # df_doe = df_doe.reset_index()
#     df_doe.columns = variables[:-1]
#     df_doe[variables[-1]] = model.predict(df_doe).round(3)
#     return df_doe

def random_generator(samples_, actual_lows, actual_highs, variables_, model):
    df_doe = pd.DataFrame(columns=variables_[:-1])
    for var in variables_[:-1]:
        df_doe[var] = np.random.uniform(actual_lows[var], actual_highs[var],
                                        samples_).round(3)
    try:
        df_doe[variables_[-1]] = model.predict(np.array(df_doe)).round(3)
    except:
        df_doe[variables_[-1]] = model.predict(df_doe).round(3)
    return df_doe

def latin_hypercube_generator(samples_, actual_lows, actual_highs, variables, model):
    # actual_lows = {'AT': [2], 'V': [30], 'AP': [993], 'RH': [30]}  # , 'PE':[425]}
    # actual_highs = {'AT': [35], 'V': [80], 'AP': [1033], 'RH': [100]}  # , 'PE':[495]}
    # variables = ['AT', 'V', 'AP', 'RH', 'PE']
    # samples_ = 100000
    # np.random.seed(5234)
    df_doe = pd.DataFrame(lhs(len(variables) - 1, samples=samples_, criterion='maximin'))
    df_doe.columns = variables[:-1]
    for col in df_doe.columns:
        df_doe[col] = [actual_lows[col]] * df_doe.shape[0] + df_doe[col] * (actual_highs[col] - actual_lows[col])
        df_doe[col] = df_doe[col].apply(lambda x: round(x, 3))
    # df_doe = df_doe.reset_index()
    df_doe.columns = variables[:-1]
    try:
        df_doe[variables_[-1]] = model.predict(np.array(df_doe)).round(3)
    except:
        df_doe[variables_[-1]] = model.predict(df_doe).round(3)
    return df_doe

def normalize(input_array):
    mean = np.mean(input_array, axis=0)
    std = np.std(input_array, axis=0)

    # scikit-learn measure to handle zeros in scale: def _handle_zeros_in_scale(scale, copy=True)
    # https://github.com/scikit-learn/scikit-learn/blob/7389dbac82d362f296dc2746f10e43ffa1615660/sklearn/preprocessing/data.py#L70
    if np.isscalar(std):
        if std == .0:
            std = 1.
    elif isinstance(std, np.ndarray):
        std = std.copy()
        std[std == 0.0] = 1.0

    data_norm = (input_array - mean) / std
    return mean, std, data_norm

df=latin_hypercube_generator(samples_, actual_lows, actual_highs, variables_, model)
print(df['pm'].min(), df['pm'].max())

-0.353 1.573


# Validate

In [162]:
#ensure repetabillity
import random
import sys
from random import randint
seed_list=[]
seed = random.randrange(sys.maxsize)
random.seed(123456)
seed_list = []
for i in range(20):
    seed_list.append(randint(0, 4292967297))
#seed_list

In [173]:
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
import lightgbm
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from pyDOE import lhs

data_path_ = r"..\..\01_Data\El_motor_Temp\pmsm_temperature_data.csv"
data = pd.read_csv(data_path_)
data_filt = data[data.profile_id==66]
X, y = data_filt.iloc[::1 ,:8], data_filt.iloc[::1 , 8].ravel()
model.fit(X, y)

size_test_set = 1000
size_train_set = 20
reference_regr = model
np.random.seed(12345)
test_data = latin_hypercube_generator(size_test_set, actual_lows, actual_highs, variables_, reference_regr)
test_data = np.array(test_data)

[12:54:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


mcs

In [174]:
seedSize = 100
actual_lows = {'ambient': -8.5, 'coolant': -1.4, 'u_d': -1.65, 'u_q': -1.8, 'motor_speed': -1.37,
              'torque': -3.3, 'i_d': -3.2, 'i_q': -3.3}  #, 'pm':[-2.6]}
actual_highs = {'ambient': 2.9, 'coolant': 2.6, 'u_d': 2.27, 'u_q': 1.79, 'motor_speed': 2.02,
              'torque': 3, 'i_d': 1.06, 'i_q': 2.9 }  #, 'pm':[2.9]}
variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']

# rr_params_v1 = {
#     'alpha': 5, 'max_iter': 4, 'normalize': False, 'solver': 'lsqr', 'tol': 0.003}
# predictor = Ridge(**rr_params_v1)
lgbmParams = {  # 'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'regression',
    'n_jobs': 6
    }
predictor = lightgbm.LGBMRegressor(**lgbmParams)

# mlp_params_v2 = {
#     'solver': 'adam', 'hidden_layer_sizes': (120, 120, 120), 'activation': 'tanh', 'tol': 1e-5, 'max_iter': 1000}
# predictor = MLPRegressor(**mlp_params_v2)

np.random.seed(seed_list[0])
train_data = np.array(random_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))
mean_data, std_data, data_all_norm = normalize(np.concatenate((train_data, test_data), axis=0))

X_train = train_data[:, :-1]
X_test = test_data[:, :-1]
y_train = train_data[:, -1].reshape((-1, 1))
y_test = test_data[:, -1].reshape((-1, 1))

# NORMALIZED DATA
X_train_norm = data_all_norm[:train_data.shape[0], :-1]
y_train_norm = data_all_norm[:train_data.shape[0], -1].reshape((-1, 1))
X_test_norm = data_all_norm[train_data.shape[0]:, :-1]
y_test_norm = data_all_norm[train_data.shape[0]:, -1].reshape((-1, 1))

#  -------->  MODEL FITTING  <------------------
predictor.fit(X_train_norm, y_train_norm.ravel())
#  -------->  PREDICTION  <------------------
mcs_y_norm = predictor.predict(X_test_norm)

# -------->  DENORMALIZATION OF PREDICTED VALUES  <------------------
mcs_y = mcs_y_norm * std_data[-1].reshape((-1, 1)) + mean_data[-1].reshape((-1, 1))


preds_mcs = np.ravel(mcs_y)
targets_mcs = np.ravel(y_test)

r2_score(targets_mcs, preds_mcs)

0.5681310675698705

lhc

In [165]:

actual_lows = {'ambient': -8.5, 'coolant': -1.4, 'u_d': -1.65, 'u_q': -1.8, 'motor_speed': -1.37,
              'torque': -3.3, 'i_d': -3.2, 'i_q': -3.3}  #, 'pm':[-2.6]}
actual_highs = {'ambient': 2.9, 'coolant': 2.6, 'u_d': 2.27, 'u_q': 1.79, 'motor_speed': 2.02,
              'torque': 3, 'i_d': 1.06, 'i_q': 2.9 }  #, 'pm':[2.9]}
variables_ = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d', 'i_q', 'pm']

# rr_params_v1 = {
#     'alpha': 5, 'max_iter': 4, 'normalize': False, 'solver': 'lsqr', 'tol': 0.003}
# predictor = Ridge(**rr_params_v1)
lgbmParams = {  # 'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'regression',
    'n_jobs': 6
    }
predictor = lightgbm.LGBMRegressor(**lgbmParams)

# mlp_params_v2 = {
#     'solver': 'adam', 'hidden_layer_sizes': (120, 120, 120), 'activation': 'tanh', 'tol': 1e-5, 'max_iter': 1000}
# predictor = MLPRegressor(**mlp_params_v2)

np.random.seed(seed_list[0])
train_data = np.array(
    latin_hypercube_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))

pool_data = np.array(
    latin_hypercube_generator(size_train_set + seedSize, actual_lows, actual_highs, variables_,
                              reference_regr))
mean_data, std_data, data_all_norm = normalize(np.concatenate((train_data, test_data), axis=0))

X_train = train_data[:, :-1]
X_test = test_data[:, :-1]
y_train = train_data[:, -1].reshape((-1, 1))
y_test = test_data[:, -1].reshape((-1, 1))

# NORMALIZED DATA
X_train_norm = data_all_norm[:train_data.shape[0], :-1]
y_train_norm = data_all_norm[:train_data.shape[0], -1].reshape((-1, 1))
X_test_norm = data_all_norm[train_data.shape[0]:, :-1]
y_test_norm = data_all_norm[train_data.shape[0]:, -1].reshape((-1, 1))

#  -------->  MODEL FITTING  <------------------
predictor.fit(X_train_norm, y_train_norm.ravel())
#  -------->  PREDICTION  <------------------
mcs_y_norm = predictor.predict(X_test_norm)

# -------->  DENORMALIZATION OF PREDICTED VALUES  <------------------
mcs_y = mcs_y_norm * std_data[-1].reshape((-1, 1)) + mean_data[-1].reshape((-1, 1))


preds_mcs = np.ravel(mcs_y)
targets_mcs = np.ravel(y_test)

r2_score(targets_mcs, preds_mcs)

0.6122953991989918

In [170]:
def rmse(true_values, predicted_values):
    n = len(true_values)
    residuals = 0
    for i in range(n):
        residuals += (true_values[i] - predicted_values[i]) ** 2.
    return np.sqrt(residuals / n)

In [176]:
committee_predictor = False

count_qbc = 0

count_qbc= 0
PoolSetSize = 360 #260#260 #400
batchSize = 5
#seedSize = 150
n_queries = 70

learner_list = []
results_qbc_t = []
startTime = datetime.now()
#est = [100, 120, 140]
# num_leaves = [6,11,16]
num_leaves = [8,11,14]
for i in range(numCommitteeMembers):
    lgbmParams = {  # 'learning_rate': 0.1,
        #'n_estimators': est[i],
        'num_leaves': num_leaves[i],
        'objective': 'regression',
        'n_jobs': 7,
        #'random_state': i*100
        }
    predictor = lightgbm.LGBMRegressor(**lgbmParams)

#         predictor = RandomForestRegressor(max_depth=num_leaves[i], 
#                                   random_state=i*100, 
#                                   #n_estimators=100, 
#                                   n_jobs=7)
    learner_list.append(ActiveLearner(estimator=predictor,
                                      # estimator=sklearn.base.clone(predictor),
                                      X_training=X_train_norm,
                                      y_training=y_train_norm.ravel(),
                                      bootstrap_init=True))
committee = CommitteeRegressor(learner_list=learner_list, query_strategy=max_std_sampling)
np.random.seed(seed_list[count_qbc])
print('seed {}'.format(count_qbc))
#     train_data = np.array(latin_hypercube_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))
#     pool_data = np.array(latin_hypercube_generator(PoolSetSize, actual_lows, actual_highs, variables_, reference_regr))
#     try:
#         fname_t = 'lhc_train_pumadyn-32_sz_' + str(seedSize) + '_randseed_' + str(seed_list[cv]) + '.npy'
#         fname_p = 'lhc_pool_pumadyn-32_sz_' + str(PoolSetSize) + '_randseed_' + str(seed_list[cv]) + '.npy'
#         train_data =  np.load(pth + fname_t)
#         pool_data = np.load(pth + fname_p)
#     except:
print('lhc file not found')
np.random.seed(seed_list[count_qbc])   
train_data = np.array(latin_hypercube_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))
#print(train_data[0])
pool_data = np.array(latin_hypercube_generator(PoolSetSize, actual_lows, actual_highs, variables_, reference_regr))


#     fname_t = 'lhc_train_pumadyn-32_sz_' + str(seedSize) + '_randseed_' + str(seed_list[cv])
#     fname_p = 'lhc_pool_pumadyn-32_sz_' + str(PoolSetSize) + '_randseed_' + str(seed_list[cv])
#     np.save(pth + fname_t, train_data)
#     np.save(pth + fname_p, pool_data)

# --------> STANDARDIZATION OF DATA SETS (subtraction of the mean, division by standard deviation) <------------------
min_data, max_data, data_all_norm = normalize(np.concatenate((train_data, pool_data, test_data), axis=0))
X_train = train_data[:, :-1]
X_pool = pool_data[:, :-1]
y_train = train_data[:, -1].reshape((-1, 1))
y_pool = pool_data[:, -1].reshape((-1, 1))
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

# NORMALIZED DATA
X_train_norm = data_all_norm[:train_data.shape[0], :-1]
y_train_norm = data_all_norm[:train_data.shape[0], -1].reshape((-1, 1))
X_pool_norm = data_all_norm[train_data.shape[0]:train_data.shape[0] + pool_data.shape[0], :-1]
y_pool_norm = data_all_norm[train_data.shape[0]:train_data.shape[0] + pool_data.shape[0], -1].reshape((-1, 1))
X_test_norm = data_all_norm[train_data.shape[0] + pool_data.shape[0]:, :-1]
y_test_norm = data_all_norm[train_data.shape[0] + pool_data.shape[0]:, -1].reshape((-1, 1))
#print(X_train_norm[0])


committee.fit(X_train_norm, y_train_norm.ravel())
 #  -------->  Initial Evaluation  <------------------
if committee_predictor:
    #  -------->  PREDICTION  <------------------
    qbc_y_norm = committee.predict(X_test_norm)
else:
    #  -------->  MODEL FITTING  <------------------
    predictor.fit(X_train_norm, y_train_norm.ravel())
    #  -------->  PREDICTION  <------------------
    qbc_y_norm = predictor.predict(X_test_norm)

pred_str = str(predictor)
pred_key = pred_str[:pred_str.find('(')]
qbc_y = max_data[-1].reshape((-1, 1)) - qbc_y_norm * (max_data[-1].reshape((-1, 1)) - min_data[-1].reshape((-1, 1)))
preds_qbc = np.ravel(qbc_y)
targets_qbc = np.ravel(y_test)

#print('QBC before, train size {}'.format(X_train_norm.shape[0]))
#print(r2_score(targets_qbc, preds_qbc))



for itter in range(n_queries+1):
    #print(itter)
    if committee_predictor:
        #  -------->  MODEL FITTING  <------------------
        committee.teach(X_train_norm, y_train_norm.ravel())
        #  -------->  PREDICTION  <------------------
        qbc_y_norm = committee.predict(X_test_norm)
    else:
        #committee.teach(X_train_norm, y_train_norm.ravel())
        #  -------->  MODEL FITTING  <------------------
        predictor.fit(X_train_norm, y_train_norm.ravel())
        #  -------->  PREDICTION  <------------------
        qbc_y_norm = predictor.predict(X_test_norm)

    # -------->  DENORMALIZATION OF PREDICTED VALUES  <------------------
    qbc_y = max_data[-1].reshape((-1, 1)) - qbc_y_norm * (max_data[-1].reshape((-1, 1)) - min_data[-1].reshape((-1, 1)))
    preds_qbc = np.ravel(qbc_y)
    targets_qbc = np.ravel(y_test)
    #print('qbc after, train size {}'.format(seedSize + num_itter*batchSize))

    r2_val = r2_score(targets_qbc, preds_qbc)
    #print('size {}, r2 {}'.format(X_train_norm.shape[0], r2_val))
    rmse_val = rmse(targets_qbc, preds_qbc)

    results_qbc_t.append({'Model': pred_key, 'CV': count_qbc, 'Training_Size': X_train_norm.shape[0],
                   'RMSE': rmse_val, 'r2_val': r2_val})

    # -------->  Pick new point  <------------------
    query_idx, query_instance = committee.query(X=X_pool_norm, n_instances=batchSize)
    X_train = np.append(X_train, X_pool[query_idx], axis=0)
    y_train = np.append(y_train, y_pool[query_idx], axis=0)
    X_train_norm = np.append(X_train_norm, X_pool_norm[query_idx], axis=0)
    y_train_norm = np.append(y_train_norm, y_pool_norm[query_idx], axis=0)
    committee.teach(X_pool_norm[query_idx],
                    y_pool_norm.ravel()[query_idx])  # train committee on newly added data
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)
    X_pool_norm = np.delete(X_pool_norm, query_idx, axis=0)
    y_pool_norm = np.delete(y_pool_norm, query_idx, axis=0)
    # print(X_pool_norm.shape)
endTime = datetime.now()
print('Total time fold {} trainSize {}: {} '.format(count_qbc, X_train_norm.shape[0], (endTime - startTime) / 60))
res_df_qbc = pd.DataFrame(results_qbc_t)
count_qbc+=1

seed 0
lhc file not found
Total time fold 0 trainSize 455: 0:00:00.166621 


In [172]:
res_df_qbc

,CV,Model,RMSE,Training_Size,r2_val
0,0,LGBMRegressor,0.410134,100,0.104381
1,0,LGBMRegressor,0.406974,105,0.118128
2,0,LGBMRegressor,0.405161,110,0.125969
3,0,LGBMRegressor,0.403579,115,0.132782
4,0,LGBMRegressor,0.401867,120,0.140121
5,0,LGBMRegressor,0.402924,125,0.135593
6,0,LGBMRegressor,0.401791,130,0.140446
7,0,LGBMRegressor,0.401334,135,0.142402
8,0,LGBMRegressor,0.400644,140,0.145349
9,0,LGBMRegressor,0.400216,145,0.147174
